# Module 04: Custom Slash Commands

**Difficulty**: ⭐⭐ Intermediate  
**Estimated Time**: 120 minutes  
**Prerequisites**: [Module 03 - Working with Skills](03_working_with_skills.ipynb)

## Learning Objectives

By the end of this module, you will be able to:

1. Create simple slash commands for common tasks
2. Use `$ARGUMENTS` for parameterized commands
3. Organize commands using namespacing
4. Understand command scopes (project vs personal)
5. Build a library of useful commands
6. Know when to use commands vs skills

---

## 1. Introduction to Slash Commands

### What are Slash Commands?

Slash commands are **custom shortcuts** that let you trigger specific prompts in Claude Code by typing `/command-name`. Think of them as customizable quick actions for your workflow.

**Example**: Instead of typing "Please review this code for security vulnerabilities", you can create `/review/security` and just type that.

### Commands vs Skills - Key Differences

| Feature | Slash Commands | Skills |
|---------|---------------|--------|
| **Activation** | Manual (`/command`) | Automatic (keyword-based) |
| **Purpose** | One-off actions, explicit triggers | Domain expertise, context awareness |
| **Complexity** | Simple prompt templates | Complex instructions + examples |
| **File Format** | `.md` files in `.claude/commands/` | `SKILL.md` in `.claude/skills/dir/` |
| **Parameters** | `$ARGUMENTS` placeholder | Context from conversation |
| **Best For** | Repeatable tasks, workflows | Automatic assistance, guidance |

### When to Use Slash Commands

Use slash commands when you:
- Need **manual control** over when an action triggers
- Have **repeatable prompts** you use frequently
- Want **quick access** to specific workflows
- Need to pass **parameters** to customize behavior

**Examples**:
- `/test unit` - Run unit tests
- `/review/code` - Code review
- `/generate/docs MyClass` - Generate documentation
- `/refactor/extract-function` - Refactoring assistance

### Command File Structure

Commands live in:
- **Project commands**: `.claude/commands/` (shared with team)
- **Personal commands**: `~/.claude/commands/` (your machine only)

Each command is a simple `.md` file containing the prompt text.

---

## 2. Creating Your First Slash Command

Let's create a simple command for code review.

### Basic Command Anatomy

A command file has:
1. **Filename**: `command-name.md` in `.claude/commands/`
2. **Content**: The prompt text Claude will receive
3. **Invocation**: Type `/command-name` in Claude Code

### Example: Code Review Command

**File**: `.claude/commands/review.md`

```markdown
Please review the code in the current file or selection for:

1. **Code Quality**: Readability, maintainability, best practices
2. **Bugs**: Potential errors, edge cases, logic issues
3. **Performance**: Inefficiencies, optimization opportunities
4. **Security**: Common vulnerabilities (SQL injection, XSS, etc.)

Provide specific, actionable feedback with code examples.
```

**Usage**: Type `/review` in Claude Code

### How It Works

When you type `/review`:
1. Claude Code reads `.claude/commands/review.md`
2. The file content becomes your prompt
3. Claude responds as if you typed the full prompt

This saves you from typing the same detailed instructions repeatedly!

---

## 3. Hands-On Section 1: Simple Commands ⭐

Let's create two useful commands for your workflow.

### Exercise 1: Create a `/review` Command

**Goal**: Create a general code review command.

**Steps**:
1. Create file `.claude/commands/review.md` in your project
2. Add the code review prompt from the example above
3. Test it by typing `/review` in Claude Code

Let's verify the commands directory exists:

In [ ]:
import os
from pathlib import Path

# Check if .claude/commands directory exists
commands_dir = Path('.claude/commands')

if commands_dir.exists():
    print(f"✅ Commands directory exists: {commands_dir}")
    
    # List existing commands
    commands = list(commands_dir.glob('*.md'))
    if commands:
        print(f"\nExisting commands ({len(commands)}):")
        for cmd in commands:
            print(f"  /{cmd.stem}")
    else:
        print("\nNo commands found yet. Let's create some!")
else:
    print(f"❌ Commands directory not found: {commands_dir}")
    print("You'll need to create it first.")

### Exercise 2: Create an `/explain` Command

**Goal**: Create a command to explain code in detail.

**File**: `.claude/commands/explain.md`

```markdown
Please explain the selected code or current file:

1. **Purpose**: What does this code do?
2. **How it works**: Step-by-step breakdown
3. **Key concepts**: Important patterns or techniques used
4. **Dependencies**: External libraries or modules required
5. **Usage examples**: How to use this code

Use clear, beginner-friendly language with code examples.
```

### Exercise 3: Test Both Commands

After creating both commands:
1. Open a code file in your project
2. Type `/review` - observe the response
3. Type `/explain` - compare the different focus

**Expected Outcome**: Both commands should trigger without typing the full prompt.

---

## 4. Parameterized Commands with $ARGUMENTS

### The Power of $ARGUMENTS

Simple commands are great, but what if you need to customize them each time?

**Enter `$ARGUMENTS`**: A special placeholder that gets replaced with whatever you type after the command.

### Syntax

```markdown
In your command file, use $ARGUMENTS where you want user input:

Please generate unit tests for the $ARGUMENTS class.
```

**Usage**: `/generate/tests Calculator`

**Result**: "Please generate unit tests for the Calculator class."

### Single vs Multiple Arguments

`$ARGUMENTS` captures **everything after the command** as a single string.

**Example**:
- Input: `/generate/tests Calculator with edge cases`
- `$ARGUMENTS` becomes: `"Calculator with edge cases"`

You can use `$ARGUMENTS` multiple times in the same command:

```markdown
Please refactor $ARGUMENTS following best practices. 
Focus on making $ARGUMENTS more readable and maintainable.
```

---

## 5. Hands-On Section 2: Parameterized Commands ⭐⭐

### Exercise 4: Create `/test` Command

**Goal**: Create a flexible testing command.

**File**: `.claude/commands/test.md`

```markdown
Please run $ARGUMENTS tests for the current project.

Steps:
1. Identify the test framework (pytest, jest, unittest, etc.)
2. Run the appropriate test command for $ARGUMENTS
3. Show the results and any failures
4. If failures exist, suggest fixes

Provide clear output and actionable next steps.
```

**Usage Examples**:
- `/test unit` - Run unit tests
- `/test integration` - Run integration tests
- `/test all` - Run all tests

### Exercise 5: Create `/generate/docs` Command

**Goal**: Generate documentation for specific code elements.

**File**: `.claude/commands/generate/docs.md`

```markdown
Please generate comprehensive documentation for $ARGUMENTS:

Include:
- **Description**: What it does
- **Parameters**: Input arguments with types
- **Returns**: Output type and description
- **Examples**: Usage examples
- **Notes**: Important considerations or edge cases

Use the appropriate documentation format (JSDoc, Sphinx, etc.) for this language.
```

**Usage**: `/generate/docs calculateTotal`

### Exercise 6: Test with Different Inputs

Try your commands with various inputs:

```
/test unit
/test integration
/generate/docs MyClass
/generate/docs the authenticate function
```

Notice how `$ARGUMENTS` adapts to different input lengths and complexity.

---

## 6. Command Namespacing

### Organizing Commands into Directories

As your command library grows, organization becomes crucial. Claude Code supports **namespacing** through directory structure.

### Namespace Structure

Instead of flat files, create subdirectories:

```
.claude/commands/
├── review/
│   ├── code.md           → /review/code
│   ├── security.md       → /review/security
│   └── performance.md    → /review/performance
├── generate/
│   ├── tests.md          → /generate/tests
│   ├── docs.md           → /generate/docs
│   └── types.md          → /generate/types
└── refactor/
    ├── extract.md        → /refactor/extract
    └── rename.md         → /refactor/rename
```

### Benefits of Namespacing

1. **Organization**: Related commands grouped together
2. **Discoverability**: Easier to find commands
3. **Clarity**: Command purpose clear from path
4. **Scalability**: Supports large command libraries
5. **Team Collaboration**: Logical structure for teams

### Naming Conventions

**Good namespace names**:
- `review/` - Code review variants
- `generate/` - Generation tasks
- `refactor/` - Refactoring operations
- `analyze/` - Analysis tasks
- `fix/` - Fix specific issues

---

## 7. Hands-On Section 3: Building a Command Library ⭐⭐⭐

### Exercise 7: Create Review Namespace

**Goal**: Build a suite of specialized review commands.

#### File 1: `.claude/commands/review/code.md`

```markdown
Please perform a comprehensive code review focusing on:

**Code Quality**:
- Variable naming and clarity
- Function size and single responsibility
- Code duplication (DRY principle)
- Comments and documentation

**Best Practices**:
- Language-specific conventions
- Design patterns usage
- Error handling
- Testing coverage

Provide specific line-by-line feedback with improvement suggestions.
```

#### File 2: `.claude/commands/review/security.md`

```markdown
Please perform a security-focused code review checking for:

**OWASP Top 10 Vulnerabilities**:
- SQL Injection
- XSS (Cross-Site Scripting)
- CSRF (Cross-Site Request Forgery)
- Insecure authentication/authorization
- Sensitive data exposure

**Additional Checks**:
- Input validation
- Output encoding
- Secrets in code (API keys, passwords)
- Dependency vulnerabilities

For each issue found, explain the risk and provide secure alternatives.
```

#### File 3: `.claude/commands/review/performance.md`

```markdown
Please analyze code performance and identify:

**Performance Issues**:
- Inefficient algorithms (time complexity)
- Memory leaks or excessive memory use
- N+1 queries (database)
- Unnecessary computations
- Blocking operations

**Optimization Opportunities**:
- Caching strategies
- Lazy loading
- Database query optimization
- Async/parallel execution

Suggest specific optimizations with estimated performance impact.
```

### Exercise 8: Create Generate Namespace

**Goal**: Commands for code generation tasks.

#### File 1: `.claude/commands/generate/tests.md`

```markdown
Please generate comprehensive unit tests for $ARGUMENTS:

**Test Coverage**:
- Happy path scenarios
- Edge cases
- Error handling
- Boundary conditions

**Test Quality**:
- Clear test names
- Arrange-Act-Assert pattern
- Mock external dependencies
- Independent, repeatable tests

Use the appropriate testing framework for this language.
```

#### File 2: `.claude/commands/generate/docs.md`

```markdown
Generate documentation for $ARGUMENTS following these guidelines:

**Documentation Elements**:
- Clear description of purpose
- Parameter documentation (types, constraints)
- Return value documentation
- Usage examples (simple and advanced)
- Error conditions and exceptions

**Format**:
- Use language-appropriate doc format (JSDoc, Sphinx, GoDoc, etc.)
- Include code examples
- Link to related functions/classes
```

#### File 3: `.claude/commands/generate/types.md`

```markdown
Please generate TypeScript types/interfaces for $ARGUMENTS:

**Requirements**:
- Accurate type definitions
- Use appropriate types (string, number, boolean, union, etc.)
- Include JSDoc comments
- Define optional vs required fields
- Use generics where appropriate

**Additional**:
- Validation logic if needed
- Example usage
```

### Exercise 9: Test Your Command Library

Let's verify the structure:

In [ ]:
from pathlib import Path

def show_command_tree(base_dir=Path('.claude/commands')):
    """Display command directory structure."""
    
    if not base_dir.exists():
        print(f"❌ Directory not found: {base_dir}")
        return
    
    print(f"📁 {base_dir}/\n")
    
    # Find all .md files
    commands = sorted(base_dir.rglob('*.md'))
    
    if not commands:
        print("No commands found.")
        return
    
    # Group by namespace
    namespaces = {}
    root_commands = []
    
    for cmd in commands:
        relative = cmd.relative_to(base_dir)
        parts = relative.parts
        
        if len(parts) == 1:
            # Root command
            root_commands.append(relative.stem)
        else:
            # Namespaced command
            namespace = parts[0]
            if namespace not in namespaces:
                namespaces[namespace] = []
            namespaces[namespace].append(relative.stem)
    
    # Display root commands
    if root_commands:
        print("Root Commands:")
        for cmd in root_commands:
            print(f"  /{cmd}")
        print()
    
    # Display namespaced commands
    if namespaces:
        print("Namespaced Commands:")
        for namespace, cmds in sorted(namespaces.items()):
            print(f"\n  {namespace}/")
            for cmd in sorted(cmds):
                print(f"    /{namespace}/{cmd}")
    
    print(f"\n✅ Total: {len(commands)} commands")

# Show the command tree
show_command_tree()

**Expected Output**: You should see a organized tree showing all your review/ and generate/ commands.

---

## 8. Real-World Command Patterns

Let's look at practical command patterns used in production workflows.

### Pattern 1: Code Review Commands

**Purpose**: Specialized reviews for different aspects

```
/review/code         - General code quality
/review/security     - Security vulnerabilities
/review/performance  - Performance optimization
/review/accessibility - Accessibility compliance
```

**Use Case**: Before pull request submission, run all review commands to catch issues early.

### Pattern 2: Documentation Generation

**Purpose**: Automated documentation creation

```
/generate/docs $ARGUMENTS      - Function/class documentation
/generate/readme               - README file generation
/generate/changelog            - CHANGELOG from git commits
/generate/api-docs             - API documentation
```

**Use Case**: Maintain up-to-date documentation without manual writing.

### Pattern 3: Test Generation

**Purpose**: Comprehensive test coverage

```
/generate/tests $ARGUMENTS     - Unit tests for specific code
/generate/integration-tests    - Integration test suite
/generate/e2e-tests            - End-to-end tests
/test/run unit                 - Execute unit tests
```

**Use Case**: Quickly achieve high test coverage for new code.

### Pattern 4: Refactoring Assistants

**Purpose**: Safe, guided refactoring

```
/refactor/extract-function $ARGUMENTS  - Extract code to function
/refactor/rename $ARGUMENTS            - Rename variable/function
/refactor/simplify                     - Simplify complex code
/refactor/modernize                    - Update to modern syntax
```

**Use Case**: Systematic code improvements with AI assistance.

### Pattern 5: Debug Helpers

**Purpose**: Troubleshooting and debugging

```
/debug/analyze $ARGUMENTS      - Analyze error or bug
/debug/trace                   - Add debug logging
/debug/explain-error           - Explain error message
/fix/bug $ARGUMENTS            - Suggest bug fixes
```

**Use Case**: Quickly diagnose and resolve issues.

---

## 9. Commands vs Skills: Decision Matrix

### When to Use Slash Commands

✅ **Use Commands when**:
- You want **manual control** over when action triggers
- The task is **repeatable** with similar steps
- You need to **pass parameters** each time
- The prompt is **straightforward** (not complex domain knowledge)
- It's a **one-off action** (review, generate, test)

**Examples**:
- `/review/security` - You decide when to run security review
- `/generate/tests Calculator` - Specify what to test
- `/refactor/extract handleSubmit` - Explicit refactoring request

### When to Use Skills

✅ **Use Skills when**:
- You want **automatic activation** based on context
- The task requires **domain expertise** or complex knowledge
- You're providing **ongoing guidance** (not one-time action)
- The skill needs **examples and patterns** for reference
- It should activate when user **mentions keywords**

**Examples**:
- `data-science-educator` - Activates when teaching pandas
- `notebook-creator` - Activates for notebook creation
- `security-expert` - Provides security guidance automatically

### Decision Tree

```
Start: Need to customize Claude Code behavior
    |
    ├─ Should it trigger automatically? 
    │   ├─ YES → Use Skill
    │   └─ NO → Continue
    |
    ├─ Need to pass different parameters each time?
    │   ├─ YES → Use Command with $ARGUMENTS
    │   └─ NO → Continue
    |
    ├─ Is it complex domain knowledge?
    │   ├─ YES → Use Skill
    │   └─ NO → Use Command
```

### Comparison Table

| Scenario | Commands | Skills | Winner |
|----------|----------|--------|--------|
| Run tests with parameter | `/test unit` | ❌ Can't pass params | ✅ Command |
| Auto-activate for ML tasks | ❌ Manual only | `ml-expert` skill | ✅ Skill |
| Code review on demand | `/review/code` | ✅ Could work | ✅ Command (manual control) |
| Teaching programming | ❌ One-time | `educator` skill | ✅ Skill (ongoing) |
| Generate docs with name | `/generate/docs Foo` | ❌ Can't specify | ✅ Command |

### Can You Use Both?

**Yes!** Commands and skills complement each other:

**Example**: Code review workflow
- **Skill**: `code-quality-expert` - Provides general code quality guidance automatically
- **Command**: `/review/security` - Explicit security review when needed

The skill gives ongoing suggestions, the command provides focused analysis on demand.

---

## 10. Best Practices for Slash Commands

### 1. Command Naming Conventions

**Good Names**:
- `review` - Clear, concise
- `generate/tests` - Namespaced, descriptive
- `fix/security` - Action + focus

**Bad Names**:
- `r` - Too short, unclear
- `do-the-thing` - Vague
- `generateTestsForMyCode` - Too long, no namespace

**Naming Rules**:
1. Use lowercase with hyphens: `review-code` not `ReviewCode`
2. Start with action verb: `generate`, `review`, `fix`, `analyze`
3. Use namespaces for related commands: `review/security`
4. Keep names under 20 characters
5. Be descriptive but concise

### 2. Writing Effective Prompts

**Good Prompt Structure**:
```markdown
Please [ACTION] for $ARGUMENTS:

**Section 1**: Specific requirement
**Section 2**: Another requirement

Output format: [Specify format]
```

**Example**:
```markdown
Please generate unit tests for $ARGUMENTS:

**Coverage**: Test happy path, edge cases, and errors
**Style**: Use Arrange-Act-Assert pattern
**Framework**: Auto-detect (pytest, jest, etc.)

Output: Complete test file with imports
```

### 3. Using $ARGUMENTS Effectively

**Do**:
- Use `$ARGUMENTS` when you need different input each time
- Place it where the dynamic content should go
- Use it multiple times if needed

**Don't**:
- Use `$ARGUMENTS` if the command never needs parameters
- Expect structured data (it's just a string)
- Rely on users typing it in a specific format

**Example - Multiple uses**:
```markdown
Refactor $ARGUMENTS to improve readability.
After refactoring $ARGUMENTS, explain the changes made.
```

### 4. Organizing Command Libraries

**Recommended Structure**:
```
.claude/commands/
├── review/          # Code review variants
├── generate/        # Generation tasks  
├── test/            # Testing commands
├── refactor/        # Refactoring helpers
├── analyze/         # Analysis tasks
└── fix/             # Bug fixing
```

**Benefits**:
- Easy to find related commands
- Clear purpose from directory name
- Scalable as library grows
- Team members can navigate easily

### 5. Project vs Personal Commands

**Project Commands** (`.claude/commands/`):  
✅ Share with team via git  
✅ Project-specific workflows  
✅ Team standards and conventions  

**Personal Commands** (`~/.claude/commands/`):  
✅ Your personal preferences  
✅ Cross-project utilities  
✅ Not committed to git  

### 6. Team Command Standards

When working in teams, establish standards:

**Document in `COMMANDS.md`**:
```markdown
# Team Slash Commands

## Review Commands
- `/review/code` - General code review
- `/review/security` - Security-focused review

## Usage Guidelines
- Run `/review/security` before every PR
- Use `/generate/tests` for new functions
```

### 7. Maintenance Tips

1. **Review periodically**: Remove unused commands
2. **Update prompts**: Improve based on experience
3. **Document purpose**: Add comments in command files
4. **Version control**: Commit commands to git
5. **Share learnings**: Tell team about useful commands

---

## 11. Practice Exercises

Now it's your turn to apply what you've learned!

### Exercise 10: Create 5 Useful Commands for Your Workflow ⭐⭐

**Goal**: Build a personal command library.

**Task**: Create 5 commands tailored to your work:

1. One for code review (specify your focus area)
2. One for documentation generation
3. One for testing
4. One for refactoring
5. One wildcard (your choice)

**Requirements**:
- At least 2 should use `$ARGUMENTS`
- At least 2 should be namespaced
- All should have clear, specific prompts

**Deliverable**: Working commands in `.claude/commands/`

### Exercise 11: Convert a Skill into a Command ⭐⭐

**Goal**: Understand the difference between skills and commands.

**Task**: 
1. Find a skill from Module 03 (or create a simple one)
2. Convert it into a slash command
3. Explain:
   - What changed?
   - What was lost in translation?
   - When would you use each version?

**Example**: Convert `code-reviewer` skill → `/review/code` command

### Exercise 12: Build a Debugging Command Set ⭐⭐⭐

**Goal**: Create a comprehensive debugging toolkit.

**Task**: Create a `debug/` namespace with:
1. `/debug/analyze $ARGUMENTS` - Analyze error/bug
2. `/debug/trace` - Add logging/tracing
3. `/debug/reproduce` - Create minimal reproduction
4. `/debug/explain-error` - Explain error messages
5. `/debug/performance` - Profile performance issues

**Bonus**: Create a `/debug/workflow` command that guides through the full debugging process.

### Exercise 13: Create Documentation Commands ⭐⭐⭐

**Goal**: Automate documentation tasks.

**Task**: Create comprehensive documentation commands:

1. `/docs/function $ARGUMENTS` - Document a function
2. `/docs/class $ARGUMENTS` - Document a class
3. `/docs/api` - Generate API documentation
4. `/docs/readme` - Update README based on code changes
5. `/docs/changelog` - Generate changelog from git history

**Requirements**:
- Commands should specify output format
- Include examples in prompts
- Handle multiple languages

---

## 12. Validation Script

Let's create a script to validate your command library:

In [ ]:
from pathlib import Path
import re

def validate_command_library(base_dir=Path('.claude/commands')):
    """Validate command library structure and quality."""
    
    print("🔍 Validating Command Library\n")
    print("=" * 50)
    
    if not base_dir.exists():
        print(f"❌ Commands directory not found: {base_dir}")
        return
    
    commands = list(base_dir.rglob('*.md'))
    
    if not commands:
        print("❌ No commands found!")
        return
    
    print(f"\n📊 Found {len(commands)} commands\n")
    
    # Quality checks
    issues = []
    parameterized = 0
    namespaced = 0
    
    for cmd_file in commands:
        relative = cmd_file.relative_to(base_dir)
        cmd_name = f"/{str(relative)[:-3]}"  # Remove .md
        
        # Read content
        content = cmd_file.read_text()
        
        # Check if parameterized
        if '$ARGUMENTS' in content:
            parameterized += 1
        
        # Check if namespaced
        if '/' in cmd_name[1:]:  # Exclude leading /
            namespaced += 1
        
        # Check file size (prompt should have substance)
        if len(content) < 50:
            issues.append(f"⚠️  {cmd_name}: Prompt too short (<50 chars)")
        
        # Check naming (lowercase, hyphens)
        parts = relative.parts
        for part in parts:
            name = part.replace('.md', '')
            if not re.match(r'^[a-z0-9-]+$', name):
                issues.append(f"⚠️  {cmd_name}: Use lowercase-with-hyphens")
                break
    
    # Summary
    print("✅ Quality Metrics:\n")
    print(f"  Total Commands: {len(commands)}")
    print(f"  Parameterized (with $ARGUMENTS): {parameterized}")
    print(f"  Namespaced: {namespaced}")
    print(f"  Root Level: {len(commands) - namespaced}")
    
    # Issues
    if issues:
        print(f"\n⚠️  Issues Found ({len(issues)}):\n")
        for issue in issues:
            print(f"  {issue}")
    else:
        print("\n✅ No issues found!")
    
    # Recommendations
    print("\n💡 Recommendations:\n")
    
    if parameterized < len(commands) * 0.3:
        print("  • Consider adding $ARGUMENTS to more commands for flexibility")
    
    if namespaced < len(commands) * 0.5:
        print("  • Use namespaces to organize related commands")
    
    if len(commands) < 5:
        print("  • Build your library! Aim for at least 5-10 useful commands")
    
    print("\n" + "=" * 50)
    print("\n✅ Validation Complete!")

# Run validation
validate_command_library()

Use this script to check your command library quality!

---

## 13. Summary

### What You've Learned

In this module, you mastered:

✅ **Slash Command Basics**
- Creating simple commands in `.claude/commands/`
- Understanding command file structure
- Manual triggering with `/command-name`

✅ **Parameterized Commands**
- Using `$ARGUMENTS` for dynamic input
- Creating flexible, reusable commands
- Handling different parameter types

✅ **Command Organization**
- Namespacing with subdirectories
- Building logical command libraries
- Project vs personal command scopes

✅ **Real-World Patterns**
- Code review command suites
- Documentation automation
- Test generation
- Debugging helpers

✅ **Commands vs Skills**
- When to use each approach
- Decision matrix for choosing
- Combining both for maximum effectiveness

✅ **Best Practices**
- Naming conventions
- Effective prompt structure
- Library organization
- Team collaboration

### Key Takeaways

1. **Commands are for manual, repeatable actions** - You control when they run
2. **Use $ARGUMENTS for flexibility** - Pass different inputs each time
3. **Organize with namespaces** - Keep related commands together
4. **Combine with skills** - Use both for complete workflows
5. **Share with your team** - Project commands improve team productivity

### Commands vs Skills Quick Reference

| Use Case | Solution |
|----------|----------|
| Manual trigger needed | ✅ Command |
| Automatic activation | ✅ Skill |
| Need parameters | ✅ Command with $ARGUMENTS |
| Complex domain knowledge | ✅ Skill |
| Simple prompt template | ✅ Command |

---

## 14. What's Next?

In **Module 05: Hooks and Automation**, you'll learn:

- 🪝 **Event-driven automation** with hooks
- ✅ **Quality gates** that run automatically
- 🛡️ **Validation hooks** to prevent mistakes
- 🔄 **Pre-commit workflows** for code quality
- ⚡ **Automated testing and linting**

Hooks take your customization to the next level by reacting to events automatically!

---

## 15. Additional Resources

### Official Documentation
- [Claude Code Slash Commands Guide](https://docs.anthropic.com/claude-code/slash-commands)
- [Command Configuration Reference](https://docs.anthropic.com/claude-code/configuration)

### Community Resources
- [Awesome Claude Code Commands](https://github.com/topics/claude-code-commands) - Community command libraries
- [Claude Code Discord](https://discord.gg/claude-developers) - Share and discover commands

### Example Command Libraries
- Development workflow commands
- Data science commands
- DevOps automation commands

### Related Modules
- **Module 03**: [Working with Skills](03_working_with_skills.ipynb)
- **Module 05**: [Hooks and Automation](05_hooks_automation.ipynb) (coming next)
- **Module 08**: [Advanced Workflows](08_advanced_workflows.ipynb) (combining everything)

---

## 🎉 Congratulations!

You've completed Module 04 and can now:
- Create powerful slash commands
- Build organized command libraries
- Use parameters for flexible commands
- Make informed decisions between commands and skills

**Next Step**: Move to Module 05 to learn about hooks and automation!

---

*Claude Code Mastery Series - Module 04 of 10*